In [1]:
from eval import get_controller
import os
import pickle as pkl
import yaml
from repr_control.agent.sac.sac_agent import SACAgent
import torch
import numpy as np
import gymnasium

In [2]:
log_path = '/home/haitong/PycharmProjects/repr_learning/repr_control/repr_control/log/sac/Ant-v4/seed_0_2024-09-08-13-23-37'

In [10]:
def collect_data(log_path):
    try:
        with open(os.path.join(os.path.dirname(log_path), 'train_params.pkl'), 'rb') as f:
            kwargs = pkl.load(f)
    except:
        with open(os.path.join(os.path.dirname(log_path), 'train_params.yaml'), 'r') as f:
            kwargs = yaml.safe_load(f)
    env =gymnasium.make(kwargs['env'])
    agent =SACAgent(**kwargs)
    agent.actor.load_state_dict(torch.load(log_path))
    states = []
    actions = []
    next_states = []
    dones = []
    
    for e in range(40):
        obs, _ = env.reset(seed=kwargs['seed'] + e)
        done = False
        env_steps = 0
        while not done:
            states.append(obs)
            action = agent.select_action(obs)
            actions.append(action)
            obs, reward, truncated, terminated, _ = env.step(action)
            next_states.append(obs)
            env_steps += 1
            done = terminated or truncated
            dones.append(done)
    states = np.array(states)
    actions = np.array(actions)
    next_states = np.array(next_states)
    dones = np.array(dones)
    dataset_name = log_path.split('/')[-1].split('.')[0]
    np.savez(dataset_name, states=states, actions=actions, next_states=next_states, dones=dones)


In [4]:
def collect_data_v2(log_path):
    try:
        with open(os.path.join(os.path.dirname(log_path), 'train_params.pkl'), 'rb') as f:
            kwargs = pkl.load(f)
    except:
        with open(os.path.join(os.path.dirname(log_path), 'train_params.yaml'), 'r') as f:
            kwargs = yaml.safe_load(f)
    env =gymnasium.make(kwargs['env'])
    agent =SACAgent(**kwargs)
    agent.actor.load_state_dict(torch.load(log_path))
    states = []
    actions = []
    next_states = []
    dones = []
    
    for e in range(200):
        obs, _ = env.reset(seed=kwargs['seed'] + e)
        done = False
        env_steps = 0
        while not done:
            states.append(obs)
            action = agent.select_action(obs)
            actions.append(action)
            obs, reward, truncated, terminated, _ = env.step(action)
            next_states.append(obs)
            env_steps += 1
            done = terminated or truncated
            dones.append(done)
    states = np.array(states)
    actions = np.array(actions)
    next_states = np.array(next_states)
    dones = np.array(dones)
    dataset_name = 'big_' + log_path.split('/')[-1].split('.')[0]
    np.savez(dataset_name, states=states, actions=actions, next_states=next_states, dones=dones)


In [8]:
policy_dir = '/home/haitong/PycharmProjects/repr_learning/repr_control/repr_control/log/sac/HalfCheetah-v4/seed_0_2024-09-04-16-56-20/actor_last.pth'

In [10]:
# for file in os.listdir(log_path):
#     if file.startswith('actor') and 'last' not in file:
collect_data_v2(policy_dir)